# The motion of intermediate-mass black holes

### Description

Intermediate-mass black holes (IMBHs), i.e. black holes with mass 100 - 10'000 Msun, are expected to form in massive star clusters via several dynamical processes, but they are extremely difficult to detect with current observational facilities (even with gravitational wave detectors). 


### Assignemnt

In this project, the students will analyze the data of N-body simulations of star clusters hosting IMBHs to estimate the distance of the IMBH from the center of mass of the star cluster. The distribution of such distances probes the effect of Brownian motion and/or other dynamical perturbations on IMBHs.

### Datasets

To be defined

### Contacts

* Michela Mapelli <michela.mapelli@unipd.it>
* Mario Pasquato <mario.pasquato@gmail.com>